# Cleaning Full Congressionally Directed Spending Dataset

source of current legislators data: https://github.com/unitedstates/congress-legislators/blob/main/README.md 

Steps taken in Excel prior to cleaning: 
1. Each of the current legislators (from above source) were given an 'index' so more easily match and parse through rows.
2. Got list of all legislators with duplicate last names. Further sorted that list by duplicate within House/Senate and within the same state.
3. Got list of unique requestors grouped by "House Requestors", "Senate Requestors", and "General" (Defense did not indicate, though those were only allowed in House). Because the appropriations data often only included last name, the unique last names were used as a key and linked with the index from the current legislators dataset. If the last names were duplicates, the dataset was filtered by state first. If there was no state provided or if the name is a duplicate from within the same state, "statedup" or "no match" was given instead as a warning to collect by hand later. 
4. Single "Requestors" columns were split using Excel's 'Text to Column' feature, with commas and semicolons used as delimiters. Each requestor was then linked with the corresponding index from the current-legislators dataset. That resulted in a structure with "Requestor One"/"Requestor One Index" and so on 
5. Occasionally, columns other than state included state abbreviations at the end of the string (for example at the end of Project or Location). Went and used an Excel formula to extract those into the State column.

In [2]:
import pandas as pd
import re

In [2]:
earmarks = pd.read_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\fulldataset.csv",encoding='utf-8-sig')
states = pd.read_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\stateabbs.csv")

### Cleaning Requestor Columns
Removing misc spaces and text from requestors

In [130]:
# ##already done -- encoding current legislators (download not properly encoded - has errors with accents in names, etc)
# legog = pd.read_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\legislators-current_OG.csv",encoding='utf-8-sig')
# legog.to_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\legislators-current_encoded.csv",encoding='utf-8-sig',index=False)

In [6]:
cols = ['Requestor One (House)','Requestor Two (House)','Requestor Three (House)','Requestor Four (House)','Requestor One (Senate)','Requestor Two (Senate)','Requestor Three (Senate)','Requestor Four (Senate)','Requestor Five (Senate)','Requestor One (General)','Requestor Two (General)']

def clean_strip(string):
    if (not pd.isnull(string)) and (string != "NaN"):
        return(str(string).strip())
    else:
        return('')

for col in cols:
    earmarks[col] = earmarks[col].map(lambda x: clean_strip(x))

### Cleaning "State" column
- Replace full state names with abbreviations
- use regular expressions to extact states embedded in Project strings

In [4]:
#replace full state names with abbreviations
stated = states.set_index("State",drop=True)
stated = stated.to_dict('dict')["Code"]

for index,row in earmarks[earmarks["State"].isin(states["State"])].iterrows():
    earmarks.loc[index,"State"] = stated.get(earmarks.loc[index,"State"])

In [5]:
##FIRST LEVEL: state in bewteen two commas

extracted = earmarks[earmarks['State'].isna()]["Project"].str.extract(r'(, [A-Z]{2},)')
# extracted[0].unique()
extracted[0] = extracted[0].str.replace(" ","")
extracted[0] = extracted[0].str.replace(",","")


for index,row in earmarks[earmarks['State'].isna()].iterrows():
    if extracted[0].loc[index] in states["Code"].values:
        earmarks.loc[index,"State"] = extracted[0].loc[index]
        
        
##SECOND LEVEL (not including second comma)
        
extracted = earmarks[earmarks['State'].isna()]["Project"].str.extract(r'(, [A-Z]{2} )')
extracted[0] = extracted[0].str.replace(" ","")
extracted[0] = extracted[0].str.replace(",","")
extracted[0].unique()


for index,row in earmarks[earmarks['State'].isna()].iterrows():
    if extracted[0].loc[index] in states["Code"].values:
        earmarks.loc[index,"State"] = extracted[0].loc[index]

In [7]:
earmarks.fillna('', inplace=True)
earmarks.to_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\fulldataset_revised.csv",encoding='utf-8-sig',index=False)

# Further Cleaning and Collapsing CDS Dataset
After above steps were completed, dataset was analyzed in excel and cleaned. Sometimes, first names were split from last names as if second requestors. Those were removed where apparent.

That cleaned dataset was then brought back here for further cleaning. The indexes were linked with the legislators-current dataset to extract full name, party, and chamber. Empty columns were dropped and data was moved left (previously, when requestors were seperated by House and Senate and General there were empty cells when requestors came from just one; in this analysis, the column seperation was removed and it was just "Requestors", as the legislators themselves now had chamber attached).

In [5]:
earmarks = pd.read_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\fulldataset_filled.csv",encoding='cp1252')
legs = pd.read_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\legislators-current_encoded.csv",encoding='utf-8-sig')

#isolate requestor columns for squeezing
cols = ['Requestor One (House)', 'index', 'Requestor Two (House)', 'index2',
       'Requestor Three (House)', 'index3', 'Requestor Four (House)', 'index4',
       'Requestor One (Senate)', 'index5', 'Requestor Two (Senate)', 'index6',
       'Requestor Three (Senate)', 'index7', 'Requestor Four (Senate)',
       'index8', 'Requestor Five (Senate)', 'index9',
       'Requestor One (General)', 'index10']

In [6]:
##"collapse" dataset so that empty cells removed and requestors info moved left (differentiaion in columns between H/S removed allowing for this). Store temporarily in 'test' dataset
def squeeze_nan(x):
    import numpy as np
    original_columns = x.index.tolist()

    squeezed = x.dropna()
    squeezed.index = [original_columns[n] for n in range(squeezed.count())]

    return squeezed.reindex(original_columns, fill_value=np.nan)

test = earmarks[cols].apply(squeeze_nan, axis=1)

In [7]:
test = test.dropna(axis=1,how='all') #remove columns with only NA values
test.columns = ['Requestor One','Requestor One Index', #rename remaining columns
 'Requestor Two','Requestor Two Index',
 'Requestor Three','Requestor Three Index',
 'Requestor Four','Requestor Four Index',
 'Requestor Five','Requestor Five Index',
 'Requestor Six','Requestor Six Index',
 'Requestor Seven','Requestor Seven Index',
 'Requestor Eight','Requestor Eight Index',
 'Requestor Nine','Requestor Nine Index']

In [8]:
earmarks.drop(cols,axis=1,inplace=True) ## drop cols from earmarks that were extracted into test dataset

In [9]:
earmarks = earmarks.join(test) #join collapsed requestors info ("test") with earmarks df
earmarks = earmarks.dropna(axis=1,how='all')

In [10]:
## get only relevant data from legislators-current dataset. convert to dict for easy linking
legs = legs[['index','full_name','bioguide_id','party','type']]
legs.set_index('index',inplace=True,drop=True)
legs_dict = legs.to_dict()

In [11]:
#for each column of requestors, use index to get name, party, and chamber from lesiglators-current dataset
vals = ['Requestor One','Requestor Two','Requestor Three','Requestor Four','Requestor Five','Requestor Six','Requestor Seven','Requestor Eight','Requestor Nine']
for val in vals:
    earmarks[val + ' Full Name'] = earmarks[val + ' Index'].map(lambda x: legs_dict['full_name'].get(int(x)) if (x != "statedup") and (x != "no match") and (not pd.isnull(x)) else x)
    earmarks[val + ' Party'] = earmarks[val + ' Index'].map(lambda x: legs_dict['party'].get(int(x)) if (x != "statedup") and (x != "no match") and (not pd.isnull(x)) else x)
    earmarks[val + ' Chamber'] = earmarks[val + ' Index'].map(lambda x: legs_dict['type'].get(int(x)) if (x != "statedup") and (x != "no match") and (not pd.isnull(x)) else x)
    earmarks[val + ' bioguide_id'] = earmarks[val + ' Index'].map(lambda x: legs_dict['bioguide_id'].get(int(x)) if (x != "statedup") and (x != "no match") and (not pd.isnull(x)) else x)

In [12]:
## ordering columns
earmarks = earmarks[['Category', 'Agency', 'Account', 'Project', 'Recipient', 'Location',
    'State', 'Budget Request', 'Additional Amount', 'Amount', 'Origination',
    'Requestor One', 'Requestor One Index','Requestor One Full Name','Requestor One bioguide_id','Requestor One Chamber','Requestor One Party',
    'Requestor Two','Requestor Two Index','Requestor Two Full Name','Requestor Two bioguide_id','Requestor Two Chamber','Requestor Two Party', 
    'Requestor Three', 'Requestor Three Index','Requestor Three Full Name','Requestor Three bioguide_id','Requestor Three Chamber','Requestor Three Party',
    'Requestor Four', 'Requestor Four Index','Requestor Four Full Name','Requestor Four bioguide_id','Requestor Four Chamber','Requestor Four Party',
    'Requestor Five','Requestor Five Index','Requestor Five Full Name','Requestor Five bioguide_id','Requestor Five Chamber','Requestor Five Party',
    'Requestor Six', 'Requestor Six Index','Requestor Six Full Name','Requestor Six bioguide_id','Requestor Six Chamber','Requestor Six Party', 
    'Requestor Seven', 'Requestor Seven Index','Requestor Seven Full Name','Requestor Seven bioguide_id','Requestor Seven Chamber','Requestor Seven Party',
    'Requestor Eight','Requestor Eight Index','Requestor Eight Full Name','Requestor Eight bioguide_id','Requestor Eight Chamber','Requestor Eight Party',
    'Requestor Nine', 'Requestor Nine Index','Requestor Nine Full Name','Requestor Nine bioguide_id','Requestor Nine Chamber','Requestor Nine Party']]

##stripping extra spaces
df_obj = earmarks.select_dtypes(['object'])
earmarks[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [21]:
#drop misc columns from sharing dataset
earmarks_forsharing = earmarks.drop(['Requestor One','Requestor One Index',
 'Requestor Two','Requestor Two Index',
 'Requestor Three','Requestor Three Index',
 'Requestor Four','Requestor Four Index',
 'Requestor Five','Requestor Five Index',
 'Requestor Six','Requestor Six Index',
 'Requestor Seven','Requestor Seven Index',
 'Requestor Eight','Requestor Eight Index',
 'Requestor Nine','Requestor Nine Index'],axis=1)

In [27]:
cols = earmarks_forsharing.columns
earmarks_forsharing.columns = [col.replace(" ","_").lower() for col in cols]

In [12]:
#save
earmarks.to_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\dataset_adjustedwithmembers.csv",encoding='utf-8-sig',index=False)
earmarks_forsharing.to_csv("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Earmarks Data\\earmarks_forsharing.csv",encoding='utf-8-sig',index=False)